# Exploring Dataset


In [ ]:
import pandas as pd
from datasets import load_dataset

train_dataset = load_dataset("tyqiangz/multilingual-sentiments", 'all', split='train')

In [ ]:
train_set = train_dataset.train_test_split()

In [ ]:
len(train_set)

2

In [ ]:
df = pd.DataFrame(train_dataset.to_dict())
df

,text,source,language,label
0,yang memerlukan pemerhatian dan tindakan serius,malaya,malay,0
1,sentiasa memikirkan dan merancang inisiatif ba...,malaya,malay,0
2,Kita akan tengok daripada pelbagai aspek supay...,malaya,malay,0
3,justeru asean perlu mengambil tindakan sebagai...,malaya,malay,0
4,"@_Niiar_ Jangan punah dulu, aku belum ke labua...",malaya,malay,1
...,...,...,...,...
270394,RT @user: #أيفون_البروفيسورمسابقة على أيفون7 أ...,sem_eval_2017,arabic,1
270395,اللهم أنت السلام ومنك السلام تباركت يا ذا الجل...,sem_eval_2017,arabic,0
270396,على وقع حمى الانتخابات الأميركية ويكليكس تكشف ...,sem_eval_2017,arabic,2
270397,"@user @user على البي سي""ويندوز 10""",sem_eval_2017,arabic,1


In [ ]:
from model import load_pretrained_gpt2_tokenizer

In [ ]:
tokenizer = load_pretrained_gpt2_tokenizer()

In [ ]:
tokenizer(df['text'][2500])

{'input_ids': [51, 316, 15042, 275, 10102, 393, 648, 300, 391, 1066, 263, 600, 993, 479, 5350, 479, 1045, 256, 461, 731, 377, 11, 256, 461, 1888, 283, 11, 256, 461, 16486, 84, 11, 256, 461, 1458, 315], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [ ]:
df['text'][270398]

'عندي وحده تشبه هاري بوتر يا انها كيوتيه😨!😭💗'

In [ ]:
from 

# Test

In [ ]:
from utils import (
    read_yaml,
    get_tokenizer_config,
    get_dataset_config,
    DEVICE,
)
from gpt_dataset import GPTDataset
from model import load_model, load_tokenizer, generate

In [ ]:
config = read_yaml('./train_config.yaml')

tokenizer_name, tokenizer_path, tokenizer_config = get_tokenizer_config(config)
tokenizer = load_tokenizer(tokenizer_name, tokenizer_path)

dataset_path, dataset_desc, dataset_config = get_dataset_config(config)
train_dataset = GPTDataset(
    filepath=dataset_path, tokenizer=tokenizer, **dataset_config
)

Token indices sequence length is longer than the specified maximum sequence length for this model (5145 > 1024). Running this sequence through the model will result in indexing errors


In [ ]:
with open('./dataset/the-verdict.txt', 'r') as F:
    file_text = F.read()
file_text[:10]

'I HAD alwa'

# Predict

In [2]:
import os
import random
from argparse import ArgumentParser
import logging

import torch
from transformers import TrainingArguments, Trainer, DataCollatorWithPadding

from utils import (
    read_yaml,
    get_model_config,
    get_tokenizer_config,
    get_split_config,
    get_dataset_config,
    get_trainer_config,
    get_prediction_config,
    get_lora_config,
    _handle_seed,
    DEVICE,
)
from sentiment_dataset import SentimentDataset
from model import (
    load_model,
    load_tokenizer,
    load_lora_model,
    predict,
    compute_metrics,
    METRICS_DICT,
)


In [4]:
config = read_yaml("./configs/predict_config_llama_lora.yaml")
# config = read_yaml("./configs/predict_config_gpt2.yaml")
# config = read_yaml("./configs/predict_config_llama.yaml")

In [5]:
tokenizer_name, tokenizer_path, tokenizer_config = get_tokenizer_config(config)
tokenizer = load_tokenizer(
    tokenizer_name=tokenizer_name,
    tokenizer_path=tokenizer_path,
    tokenizer_config=tokenizer_config,
)

In [6]:
dataset_desc, (train_split_config, val_split_config, test_split_config) = (
    get_split_config(config)
)

test_dataset_path, test_dataset_language, test_dataset_config = (
    get_dataset_config(test_split_config)
)
test_dataset = SentimentDataset(
    dataset_path=test_dataset_path,
    dataset_language=test_dataset_language,
    split_type="test",
    tokenizer=tokenizer,
    **test_dataset_config,
)

In [7]:
model_name, model_path, base_model_path, model_config = get_model_config(config)
model_config.update(dict(pad_token_id=tokenizer.pad_token_id))
model = load_model(
    model_string=model_name,
    model_path=model_path,
    base_model_path=base_model_path,
    model_config=model_config,
)
model = model.to(device=DEVICE)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some weights of LlamaForSequenceClassification were not initialized from the model checkpoint at meta-llama/Llama-2-7b-hf and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
prompt_samples = [
    "this is a bad product",
    "this the nice product",
    "this is a best product",
    "how are you?"
]

In [9]:
predcitions = predict(
    model, tokenizer, prompt_samples, device=DEVICE,
)
predcitions

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


/home/hmankodi/bitsandbytes/bitsandbytes/nn/modules.py:451: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(


(array([2, 0, 0, 1]), ['negative', 'positive', 'positive', 'neutral'])

In [10]:
import evaluate
from model import METRICS_DICT

In [11]:
data_collator = DataCollatorWithPadding(
    tokenizer=tokenizer,
    padding=test_dataset_config.get("padding", True),
    return_tensors="pt",
)

trainer = Trainer(
    model,
    data_collator=data_collator,
    compute_metrics=lambda x: (compute_metrics(x, METRICS_DICT)),
)

Detected kernel version 4.18.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [12]:
trainer.evaluate(test_dataset)

{'eval_loss': 1.0556640625,
 'eval_model_preparation_time': 0.0045,
 'eval_accuracy': 0.68766,
 'eval_auc': 0.85569,
 'eval_runtime': 690.2538,
 'eval_samples_per_second': 20.956,
 'eval_steps_per_second': 2.621}

In [11]:
def count_lora_params(model):
    """
    Prints and counts the total trainable parameters for LoRA layers in a model.
    """
    total_params = 0
    print("LoRA Trainable Parameters:\n")
    for name, param in model.named_parameters():
        if param.requires_grad:
            num_params = param.numel()
            total_params += num_params
            print(f"{name}: {num_params} parameters")
    
    print(f"\nTotal LoRA trainable parameters: {total_params}")
    return total_params

In [13]:
sum(i.numel() for i in model.parameters())

3409342464

In [14]:
model.print_trainable_parameters()

trainable params: 12,288 || all params: 6,647,345,152 || trainable%: 0.0002
